# Mistral Fine-tuning API

Check out the docs: https://docs.mistral.ai/capabilities/finetuning/

In [1]:
!pip install mistralai pandas



[notice] A new release of pip is available: 24.2 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


## Prepare the dataset

In this example, let’s use the ultrachat_200k dataset. We load a chunk of the data into Pandas Dataframes, split the data into training and validation, and save the data into the required jsonl format for fine-tuning.

In [21]:
import pandas as pd
df = pd.read_csv('C:\\Users\\user\\OneDrive\\Bureau\\mistral-gagnant\\fine_tune\\dataset\\amazon.csv')
df_train=df.sample(frac=0.995,random_state=200)
df_eval=df.drop(df_train.index)

df_train.to_json("ultrachat_chunk_train.jsonl", orient="records", lines=True)
df_eval.to_json("ultrachat_chunk_eval.jsonl", orient="records", lines=True)

SyntaxError: (unicode error) 'unicodeescape' codec can't decode bytes in position 2-3: truncated \UXXXXXXXX escape (3563495986.py, line 2)

In [13]:
!ls -lh

total 147M
-rw-r--r-- 1 root root 3.4K Nov 23 10:39 reformat_data.py
drwxr-xr-x 1 root root 4.0K Nov 21 14:25 sample_data
-rw-r--r-- 1 root root 698K Nov 23 10:43 ultrachat_chunk_eval.jsonl
-rw-r--r-- 1 root root 146M Nov 23 10:43 ultrachat_chunk_train.jsonl


## Reformat dataset
If you upload this ultrachat_chunk_train.jsonl to Mistral API, you might encounter an error message “Invalid file format” due to data formatting issues. To reformat the data into the correct format, you can download the reformat_dataset.py script and use it to validate and reformat both the training and evaluation data:

In [14]:
# download the validation and reformat script
!wget https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py

--2024-11-23 10:44:04--  https://raw.githubusercontent.com/mistralai/mistral-finetune/main/utils/reformat_data.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.110.133, 185.199.109.133, 185.199.111.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|185.199.110.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 3381 (3.3K) [text/plain]
Saving to: ‘reformat_data.py.1’

reformat_data.py.1  100%[===================>]   3.30K  --.-KB/s    in 0s      

2024-11-23 10:44:04 (41.7 MB/s) - ‘reformat_data.py.1’ saved [3381/3381]



In [15]:
# validate and reformat the training data
!python reformat_data.py ultrachat_chunk_train.jsonl

Skipped 3674th sample
Skipped 9176th sample
Skipped 10559th sample
Skipped 13293th sample
Skipped 13973th sample
Skipped 15219th sample


In [16]:
# validate the reformat the eval data
!python reformat_data.py ultrachat_chunk_eval.jsonl

In [17]:
df_train.iloc[3674]['messages']

array([{'content': 'What are the dimensions of the cavity, product, and shipping box of the Sharp SMC1662DS microwave?: With innovative features like preset controls, Sensor Cooking and the Carousel® turntable system, the Sharp® SMC1662DS 1.6 cu. Ft. Stainless Steel Carousel Countertop Microwave makes reheating your favorite foods, snacks and beverages easier than ever. Use popcorn and beverage settings for one-touch cooking. Express Cook allows one-touch cooking up to six minutes. The convenient and flexible "+30 Sec" key works as both instant start option and allows you to add more time during cooking.\nThe Sharp SMC1662DS microwave is a bold design statement in any kitchen. The elegant, grey interior and bright white, LED interior lighting complements the stainless steel finish of this premium appliance.\nCavity Dimensions (w x h x d): 15.5" x 10.2" x 17.1"\nProduct Dimensions (w x h x d): 21.8" x 12.8" x 17.7"\nShipping Dimensions (w x h x d) : 24.4" x 15.0" x 20.5"', 'role': 'user

## Upload dataset

In [23]:
from mistralai import Mistral
import os

api_key = "tiR6ZozP4kC4J3GLLdjkr990lwXwFnOO"

client = Mistral(api_key=api_key)

ultrachat_chunk_train = client.files.upload(file={
    "file_name": "ultrachat_chunk_train.jsonl",
    "content": open("ultrachat_chunk_train.jsonl", "rb"),
})
ultrachat_chunk_eval = client.files.upload(file={
    "file_name": "ultrachat_chunk_eval.jsonl",
    "content": open("ultrachat_chunk_eval.jsonl", "rb"),
})

In [24]:
import json
def pprint(obj):
    print(json.dumps(obj.dict(), indent=4))

In [25]:
pprint(ultrachat_chunk_train)

{
    "id": "c6a8fec2-a19d-415f-ad70-c78e95325f1d",
    "object": "file",
    "bytes": 121379382,
    "created_at": 1732359131,
    "filename": "ultrachat_chunk_train.jsonl",
    "purpose": "fine-tune",
    "sample_type": "instruct",
    "source": "upload",
    "num_lines": 28156
}


In [26]:
pprint(ultrachat_chunk_eval)

{
    "id": "0d06d17b-587c-4212-81b8-82153a0e2a94",
    "object": "file",
    "bytes": 596255,
    "created_at": 1732359132,
    "filename": "ultrachat_chunk_eval.jsonl",
    "purpose": "fine-tune",
    "sample_type": "instruct",
    "source": "upload",
    "num_lines": 142
}


## Create a fine-tuning job

In [27]:
created_jobs = client.fine_tuning.jobs.create(
    model="open-mistral-7b",
    training_files=[{"file_id": ultrachat_chunk_train.id, "weight": 1}],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters={
    "training_steps": 10,
    "learning_rate":0.0001
    },
    auto_start=True
)
created_jobs

JobOut(id='0b0c7fe9-c539-4c83-b525-6fc25392c014', auto_start=True, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=None, fim_ratio=None, seq_len=32768), model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1732359149, modified_at=1732359149, training_files=['c6a8fec2-a19d-415f-ad70-c78e95325f1d'], validation_files=['0d06d17b-587c-4212-81b8-82153a0e2a94'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=None, cost=0.0, cost_currency=None, train_tokens_per_step=None, train_tokens=None, data_tokens=None, estimated_start_time=None))

In [28]:
pprint(created_jobs)

{
    "id": "0b0c7fe9-c539-4c83-b525-6fc25392c014",
    "auto_start": true,
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "warmup_fraction": 0.05,
        "epochs": null,
        "fim_ratio": null,
        "seq_len": 32768
    },
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1732359149,
    "modified_at": 1732359149,
    "training_files": [
        "c6a8fec2-a19d-415f-ad70-c78e95325f1d"
    ],
    "validation_files": [
        "0d06d17b-587c-4212-81b8-82153a0e2a94"
    ],
    "fine_tuned_model": null,
    "suffix": null,
    "integrations": [],
    "trained_tokens": null,
    "repositories": [],
    "metadata": {
        "expected_duration_seconds": null,
        "cost": 0.0,
        "cost_currency": null,
        "train_tokens_per_step": null,
        "train_tokens": null,
        "data_tokens": null,
        "estimated_start_time": null
    }
}


In [29]:
jobs = client.fine_tuning.jobs.list()
print(jobs)

total=5 data=[JobOut(id='0b0c7fe9-c539-4c83-b525-6fc25392c014', auto_start=True, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=0.0431941570306258, fim_ratio=None, seq_len=32768), model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1732359149, modified_at=1732359152, training_files=['c6a8fec2-a19d-415f-ad70-c78e95325f1d'], validation_files=['0d06d17b-587c-4212-81b8-82153a0e2a94'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=140, cost=2.5, cost_currency='EUR', train_tokens_per_step=131072, train_tokens=1310720, data_tokens=30344845, estimated_start_time=1732360978)), JobOut(id='f0c1608f-cd6d-400b-bf87-fa52ceaa1530', auto_start=False, hyperparameters=TrainingParameters(training_steps=12, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=10.0, fim_ratio=None, seq_len=327

In [30]:
retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
retrieved_jobs

DetailedJobOut(id='0b0c7fe9-c539-4c83-b525-6fc25392c014', auto_start=True, hyperparameters=TrainingParameters(training_steps=10, learning_rate=0.0001, weight_decay=0.1, warmup_fraction=0.05, epochs=0.0431941570306258, fim_ratio=None, seq_len=32768), model='open-mistral-7b', status='QUEUED', job_type='FT', created_at=1732359149, modified_at=1732359152, training_files=['c6a8fec2-a19d-415f-ad70-c78e95325f1d'], validation_files=['0d06d17b-587c-4212-81b8-82153a0e2a94'], OBJECT='job', fine_tuned_model=None, suffix=None, integrations=[], trained_tokens=None, repositories=[], metadata=JobMetadataOut(expected_duration_seconds=140, cost=2.5, cost_currency='EUR', train_tokens_per_step=131072, train_tokens=1310720, data_tokens=30344845, estimated_start_time=1732360980), events=[EventOut(name='status-updated', created_at=1732359152, data={'status': 'QUEUED'}), EventOut(name='status-updated', created_at=1732359152, data={'status': 'VALIDATED'}), EventOut(name='status-updated', created_at=1732359152,

In [32]:
import time

retrieved_job = client.fine_tuning.jobs.get(job_id = created_jobs.id)
while retrieved_job.status in ["RUNNING", "QUEUED"]:
    retrieved_job = client.fine_tuning.jobs.get(job_id = created_jobs.id)
    pprint(retrieved_job)
    print(f"Job is {retrieved_job.status}, waiting 10 seconds")
    time.sleep(10)



{
    "id": "0b0c7fe9-c539-4c83-b525-6fc25392c014",
    "auto_start": true,
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001,
        "weight_decay": 0.1,
        "warmup_fraction": 0.05,
        "epochs": 0.0431941570306258,
        "fim_ratio": null,
        "seq_len": 32768
    },
    "model": "open-mistral-7b",
    "status": "QUEUED",
    "job_type": "FT",
    "created_at": 1732359149,
    "modified_at": 1732359152,
    "training_files": [
        "c6a8fec2-a19d-415f-ad70-c78e95325f1d"
    ],
    "validation_files": [
        "0d06d17b-587c-4212-81b8-82153a0e2a94"
    ],
    "fine_tuned_model": null,
    "suffix": null,
    "integrations": [],
    "trained_tokens": null,
    "repositories": [],
    "metadata": {
        "expected_duration_seconds": 140,
        "cost": 2.5,
        "cost_currency": "EUR",
        "train_tokens_per_step": 131072,
        "train_tokens": 1310720,
        "data_tokens": 30344845,
        "estimated_start_time": 17

KeyboardInterrupt: 

In [ ]:
# List jobs
jobs = client.fine_tuning.jobs.list()
pprint(jobs)

{
    "total": 32,
    "data": [
        {
            "id": "20178c3c-d75b-428e-b20d-7d39aa2b7468",
            "auto_start": true,
            "hyperparameters": {
                "training_steps": 10,
                "learning_rate": 0.0001,
                "epochs": 0.0431941570306258,
                "fim_ratio": null
            },
            "model": "open-mistral-7b",
            "status": "SUCCESS",
            "job_type": "FT",
            "created_at": 1721405548,
            "modified_at": 1721405693,
            "training_files": [
                "ec5af16a-77fe-4e14-ad09-47ead2848ce6"
            ],
            "validation_files": [
                "d0c643a2-a57c-4031-bda7-5c9d6c3ec3e4"
            ],
            "fine_tuned_model": "ft:open-mistral-7b:b6e34a5e:20240719:20178c3c",
            "suffix": null,
            "integrations": [],
            "trained_tokens": 1310720,
            "repositories": [],
            "metadata": {
                "expected_duration_s

In [ ]:
# Retrieve a jobs
retrieved_jobs = client.fine_tuning.jobs.get(job_id = created_jobs.id)
pprint(retrieved_jobs)


{
    "id": "20178c3c-d75b-428e-b20d-7d39aa2b7468",
    "auto_start": true,
    "hyperparameters": {
        "training_steps": 10,
        "learning_rate": 0.0001,
        "epochs": 0.0431941570306258,
        "fim_ratio": null
    },
    "model": "open-mistral-7b",
    "status": "SUCCESS",
    "job_type": "FT",
    "created_at": 1721405548,
    "modified_at": 1721405693,
    "training_files": [
        "ec5af16a-77fe-4e14-ad09-47ead2848ce6"
    ],
    "validation_files": [
        "d0c643a2-a57c-4031-bda7-5c9d6c3ec3e4"
    ],
    "fine_tuned_model": "ft:open-mistral-7b:b6e34a5e:20240719:20178c3c",
    "suffix": null,
    "integrations": [],
    "trained_tokens": 1310720,
    "repositories": [],
    "metadata": {
        "expected_duration_seconds": 120,
        "cost": 2.6214,
        "cost_currency": "USD",
        "train_tokens_per_step": 131072,
        "train_tokens": 1310720,
        "data_tokens": 30344845,
        "estimated_start_time": null
    },
    "events": [
        {
  

## Use a fine-tuned model

In [ ]:
chat_response = client.chat.complete(
    model = retrieved_jobs.fine_tuned_model,
    messages = [{"role":'user', "content":'What is the best French cheese?'}]
)

In [ ]:
pprint(chat_response)

{
    "id": "1fac96713fd74799922712e34e009f81",
    "object": "chat.completion",
    "model": "ft:open-mistral-7b:b6e34a5e:20240719:20178c3c",
    "usage": {
        "prompt_tokens": 10,
        "completion_tokens": 73,
        "total_tokens": 83
    },
    "created": 1721405725,
    "choices": [
        {
            "index": 0,
            "finish_reason": "stop",
            "message": {
                "content": "There isn't a single \"best\" French cheese as there are hundreds of different types of cheese to choose from, each with its unique taste and texture. Some popular French cheeses include Brie, Camembert, Roquefort, Comt\u00e9, and Ch\u00e8vre. Try different cheeses to find out the one you like best!",
                "tool_calls": null,
                "prefix": false,
                "role": "assistant"
            }
        }
    ]
}


## Integration with Weights and Biases
We can also offer support for integration with Weights & Biases (W&B) to monitor and track various metrics and statistics associated with our fine-tuning jobs. To enable integration with W&B, you will need to create an account with W&B and add your W&B information in the “integrations” section in the job creation request:



In [ ]:
client.fine_tuning.jobs.create(
    model="open-mistral-7b",
    training_files=[{"file_id": ultrachat_chunk_train.id, "weight": 1}],
    validation_files=[ultrachat_chunk_eval.id],
    hyperparameters={"training_steps": 10, "learning_rate": 0.0001},
    integrations=[
        {
            "project": "<value>",
            "api_key": "<value>",
        }
    ]
)